<a href="https://colab.research.google.com/github/Nareshedagotti/RAG/blob/main/Agentic_Rag.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install langchain langchain_community qdrant_client groq duckduckgo-search agno

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.0 MB/s eta 0:00:00


Load Web Document

In [4]:
from langchain_community.document_loaders import WebBaseLoader

urls = [
    "https://en.wikipedia.org/wiki/2025_Indian_Premier_League"
]

docs = [WebBaseLoader(url).load() for url in urls]

Split Documents

In [6]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

docs_list = [item for sublist in docs for item in sublist]

text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=200, chunk_overlap=50
)
doc_splits = text_splitter.split_documents(docs_list)

In [22]:
doc_splits

[Document(metadata={'source': 'https://en.wikipedia.org/wiki/2025_Indian_Premier_League', 'title': '2025 Indian Premier League - Wikipedia', 'language': 'en'}, page_content='2025 Indian Premier League - Wikipedia\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nJump to content\n\n\n\n\n\n\n\nMain menu\n\n\n\n\n\nMain menu\nmove to sidebar\nhide\n\n\n\n\t\tNavigation\n\t\n\n\nMain pageContentsCurrent eventsRandom articleAbout WikipediaContact us\n\n\n\n\n\n\t\tContribute\n\t\n\n\nHelpLearn to editCommunity portalRecent changesUpload fileSpecial pages\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nSearch\n\n\n\n\n\n\n\n\n\n\n\nSearch\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nAppearance\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nDonate\n\nCreate account\n\nLog in\n\n\n\n\n\n\n\n\nPersonal tools\n\n\n\n\n\nDonate Create account Log in\n\n\n\n\n\n\t\tPages for logged out editors learn more\n\n\n\nContributionsTalk'),
 Document(metadata={'source': 'https://en.wikipedia.org/wiki/2025_Ind

Generate Embeddings

In [7]:
from langchain.embeddings import HuggingFaceBgeEmbeddings

def generate_embeddings(doc_splits):
    model_name = "BAAI/bge-small-en-v1.5"
    model_kwargs = {'device': 'cpu'}
    encode_kwargs = {'normalize_embeddings': False}
    hf = HuggingFaceBgeEmbeddings(
        model_name=model_name,
        model_kwargs=model_kwargs,
        encode_kwargs=encode_kwargs
    )
    return hf,doc_splits

embeddings, doc_splits = generate_embeddings(doc_splits)


<ipython-input-7-f83f04b84c11>:7: LangChainDeprecationWarning: The class `HuggingFaceBgeEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  hf = HuggingFaceBgeEmbeddings(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/94.8k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Store Embeddings in VectorDB (Qdrant)

In [9]:
from langchain.vectorstores import Qdrant

qdrant = Qdrant.from_documents(
    doc_splits,
    embeddings,
    location=":memory:",
    collection_name="my_documents",
)

Vector Query & Retrieval

In [10]:
#converting query into embeddings
def query_to_embeddings(query, embedding_model):
    query_embedding = embeddings.embed_query(query)
    return query_embedding

In [11]:
#retrieving top matched context from vectordb
def retrieve_top_matches(vector_db, query_embedding):
    top_matches = vector_db.similarity_search_by_vector(query_embedding, k=5)
    return top_matches

Context Relevance Check Using Groq LLM

In [20]:
from groq import Groq
def is_context_relevant_llm(context, query, api_key=os.environ["GROQ_API_KEY"]):
    prompt = f"""
You are a context relevance checker.

Given the context below and a query, decide if the context is helpful in answering the query.

Context:
{context}

Query:
{query}

Answer only YES or NO, and explain briefly.

Example format:
YES: The context discusses football events in 2025.

NO: The context is about cricket and unrelated to football.
"""
    client = Groq(api_key=api_key)
    chat_completion = client.chat.completions.create(
        messages=[{"role": "user", "content": prompt}],
        model="gemma2-9b-it"
    )
    output = chat_completion.choices[0].message.content.strip()
    return output.startswith("YES"), output

Agent Setup for Web Search (Agno Agent)


In [16]:
from agno.agent import Agent
from agno.models.groq import Groq as AgnoGroq
from agno.tools.duckduckgo import DuckDuckGoTools
import os

os.environ["GROQ_API_KEY"] = "your_api_key"

web_agent = Agent(
    name="Web Agent",
    role="Search the web for current information",
    model=AgnoGroq(id="llama-3.3-70b-versatile"),
    tools=[DuckDuckGoTools()],
    instructions="Always include sources",
    markdown=True,
)

def get_web_context(query):
    response = web_agent.run(query)
    return response.content.strip()

Final Answer from LLM Based on Context

In [17]:
def final_llm_fusion(query, context, api_key=os.environ["GROQ_API_KEY"]):
    prompt = f"Context:\n{context}\n\nBased on the above context, answer the following:\nQuestion: {query}\nAnswer:"

    client = Groq(api_key=api_key)
    chat_completion = client.chat.completions.create(
        messages=[{"role": "user", "content": prompt}],
        model="gemma2-9b-it"
    )
    return chat_completion.choices[0].message.content.strip()

In [21]:
query = "What are the highlights of football 2025?"
query_embedding = embeddings.embed_query(query)
top_matches = retrieve_top_matches(qdrant, query_embedding)
doc_context = "\n\n".join([match.page_content for match in top_matches])

# Step 2: Check relevance using LLM
is_relevant, explanation = is_context_relevant_llm(doc_context, query)

if is_relevant:
    print("✅ Using document context.")
    final_response = final_llm_fusion(query, doc_context)
else:
    print("⚠️ Document context not relevant:", explanation)
    print("🌐 Using web search...")

    web_context = get_web_context(query)
    final_response = final_llm_fusion(query, web_context)

print("🧠 Final Answer:\n", final_response)

⚠️ Document context not relevant: NO: The context is about the 2025 Indian Premier League, a cricket tournament.
🌐 Using web search...
🧠 Final Answer:
 The highlights of football in 2025 mentioned are:

* **NFL Wild Card Weekend 2025**: The Commanders win and the Eagles dominant win with top playoff moments.
* **2025 Pro Bowl Games**: Notably, Houston Texans running back Joe Mixon catching a 21-yard touchdown pass.
* **Notre Dame Football Highlights**: The offense winning the 2025 Blue-Gold Game.
* **2025 NFL Honors**: Winners, snubs, top moments, and red carpet highlights.
* **NFC vs. AFC Highlights 2025 Pro Bowl**: A cinematic recap of Super Bowl XLIX between the New England Patriots and Seattle Seahawks. 


Let me know if you'd like more information about any specific highlight!
